In [71]:
from typing import List, Text, Tuple, Union
from pprint import pprint

from pyvi import ViTokenizer
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer, MultiLabelBinarizer, FunctionTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sentence_transformers import SentenceTransformer
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from emandai.utils import load_data_from_botid
from tqdm import tqdm

from va.src.utils.data import load_va_data

# First Look

## Load Data

In [45]:
PATH = "../data/trungquan/trainset.xlsx"
X, y = load_va_data(PATH)

## Preprocess Text

In [46]:
def preprocess(texts):
    """ Word Segmentation """
    tokenize = lambda x: ViTokenizer.tokenize(x)

    return [tokenize(text) for text in texts]

In [47]:
X = preprocess(X)

## Encode Label

In [48]:
lb = LabelBinarizer()
y = lb.fit_transform(y)
pprint(lb.classes_)
pprint(y)

array(['C2A_TUVAN', 'C2B_GLS', 'C3_KQT'], dtype='<U9')
array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0]])


In [49]:
def encode_label(labels):
    lb = LabelBinarizer()
    lb.fit(labels)
    return lb

## Text Embedding

### TF-IDF

In [222]:
tfidf = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", ngram_range=(1, 2))

In [223]:
tfidf_X = tfidf.fit_transform(X)

In [224]:
len(tfidf.vocabulary_)

1281

### Vietnamese SBERT

In [229]:
sbert = SentenceTransformer('keepitreal/vietnamese-sbert')

In [230]:
sbert_X = sbert.encode(X)

In [231]:
sbert_X.shape

(396, 768)

### PhoBERT

In [331]:
phobert = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base")
photokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['

In [50]:
def phobert_encode(text: Union[Text, List[Text]],
                   phobert,
                   photokenizer) -> np.ndarray:
    """ Get sentence embedding from [CLS] token of the given text.
    reference: https://discuss.huggingface.co/t/how-to-get-cls-embeddings-from-bertfortokenclassification-model/9276/3
    """
    embeddings = None

    encoded_text = photokenizer(text, padding=True, truncation=True, return_tensors="pt")
    text_ids = encoded_text["input_ids"]
    with torch.no_grad():
        outputs = phobert(text_ids, output_hidden_states=True)
        last_hidden_state = outputs.hidden_states[-1]
        embeddings = last_hidden_state[:, 0, :]

        if isinstance(embeddings, torch.Tensor):
            embeddings = embeddings.detach().cpu().numpy()

        return embeddings

In [52]:
phobert_X = phobert_encode(X, phobert, photokenizer)

In [53]:
phobert_X.shape

(396, 768)

## Classifier

### Logistic Regression

In [451]:
clf = {}
clf["tfidf"] = OneVsRestClassifier(LogisticRegression()).fit(tfidf_X, y)
clf["sbert"] = OneVsRestClassifier(LogisticRegression()).fit(sbert_X, y)
clf["phobert"] = OneVsRestClassifier(LogisticRegression()).fit(phobert_X, y)

/home/buihuy/Intent-Detection/venv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/home/buihuy/Intent-Detection/venv/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stabl

In [ ]:
""" Inference """
def infer(text: Text,
          embedding_type: Text = "tfidf",
          with_prob: bool = False
    ) -> Union[np.ndarray, Tuple[np.ndarray, np.ndarray]]:
    pred = []

    if not isinstance(text, List):
        text = [text]

    if embedding_type == "tfidf":
        vectorizer = tfidf
        _clf = clf["tfidf"]
    elif embedding_type == "sbert":
        vectorizer = sbert
        _clf = clf["sbert"]
    elif embedding_type == "phobert":
        vectorizer = phobert
        _clf = clf["phobert"]
    else:
        raise ValueError

    # preprocess
    text = preprocess(text)

    # vectorize
    if embedding_type == "tfidf":
        features = vectorizer.transform(text)
    elif embedding_type == "sbert":
        features = vectorizer.encode(text)
    elif embedding_type == "phobert":
        features = phobert_encode(text)
    else:
        raise ValueError

    # classify
    pred = _clf.predict(features)

    if with_prob:
        prob = _clf.predict_proba(features)
        prob = np.max(prob, axis=1)
        return pred, prob

    return pred

#### Features

##### TF-IDF

In [242]:
def test1(text):
    pred, prob = infer(text, embedding_type="tfidf", with_prob=True)
    pprint(pred)
    pprint(lb.inverse_transform(pred))
    pprint(prob)

In [243]:
text = "em có thể diễn giải chi tiết hơn được không"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.34704396])


In [246]:
text = "hay là chút nữa liên lạc lại em nhé"
test1(text)

array([[0, 1, 0]])
array(['C2B_GLS'], dtype='<U9')
array([0.51182048])


In [248]:
text = "cảm ơn em nhưng chắc là anh không hứng thú"
test1(text)

array([[0, 0, 1]])
array(['C3_KQT'], dtype='<U9')
array([0.62048898])


In [251]:
text = "anh nghĩ là em nên dừng lại"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.46300921])


In [253]:
text = "dạ không ạ"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.46125805])


In [257]:
text = "không đâu chị ơi"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.36902174])


In [265]:
text = "ừ không, mình đang làm rồi bạn nha"
test1(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.38223752])


##### SBERT

In [244]:
def test2(text):
    pred, prob = infer(text, embedding_type="sbert", with_prob=True)
    pprint(pred)
    pprint(lb.inverse_transform(pred))
    pprint(prob)

In [245]:
text = "em có thể diễn giải chi tiết hơn được không"
test2(text)

array([[1, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.69315568])


In [247]:
text = "hay là chút nữa liên lạc lại em nhé"
test2(text)

array([[0, 1, 0]])
array(['C2B_GLS'], dtype='<U9')
array([0.95827348])


In [249]:
text = "cảm ơn em nhưng chắc là anh không hứng thú"
test2(text)

array([[0, 0, 1]])
array(['C3_KQT'], dtype='<U9')
array([0.77097445])


In [252]:
text = "anh nghĩ là em nên dừng lại"
test2(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.21407058])


In [256]:
text = "dạ không ạ"
test2(text)

array([[1, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.89481114])


In [258]:
text = "không đâu chị ơi"
test2(text)

array([[0, 0, 1]])
array(['C3_KQT'], dtype='<U9')
array([0.66558446])


In [267]:
text = "ừ không, mình đang làm rồi bạn nha"
test2(text)

array([[0, 0, 1]])
array(['C3_KQT'], dtype='<U9')
array([0.80773914])


##### PhoBERT

In [455]:
def test3(text):
    pred, prob = infer(text, embedding_type="phobert", with_prob=True)
    pprint(pred)
    pprint(lb.inverse_transform(pred))
    pprint(prob)

In [456]:
text = "em có thể diễn giải chi tiết hơn được không"
test3(text)

array([[1, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.59992224])


In [457]:
text = "hay là chút nữa liên lạc lại em nhé"
test3(text)

array([[0, 0, 0]])
array(['C2A_TUVAN'], dtype='<U9')
array([0.37629714])


# Benchmark

## Load Data

In [54]:
botid = {
    "sunlaw": "6268f7e49f455cd4ea292d88",
    "lamhaian": "6268f8049f455c653b292e29",
    "giang.nguyen": "6268f7f89f455cd9eb292df4"
}

### Training Set

In [55]:
# load data
data = {k: load_data_from_botid(v) for k, v in botid.items()}

INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token
INFO:oauth2client.transport:Attempting refresh to obtain initial access_token
INFO:oauth2client.client:Refreshing access_token


### Test set

## Preprocess Data

In [56]:
# preprocess
def preprocess(data: pd.DataFrame) -> pd.DataFrame:
    """ Text preprocessing
    """
    # remove duplicate
    data = data.drop_duplicates("Sentence")

    # lower
    data["Sentence"] = data["Sentence"].map(lambda x: x.lower())
    data.head(n=10)

    # word segmenation
    data["Sentence"] = data["Sentence"].map(ViTokenizer.tokenize)

    return data

In [57]:
preprocessed_data = {k: preprocess(v) for k, v in data.items()}

## Get Features

In [32]:
""" Inititialize featurizers 
"""
tfidf = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", ngram_range=(1, 2))
sbert = SentenceTransformer('keepitreal/vietnamese-sbert')
phobert = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base")
photokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: keepitreal/vietnamese-sbert
INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a B

In [72]:
def get_features(X: Union[Text, List[Text]],
                 embedding_type: Text) -> np.ndarray:
    if embedding_type == "tfidf":
        X = tfidf.fit_transform(X).toarray()
    elif embedding_type == "sbert":
        X = sbert.encode(X)
    elif embedding_type == "phobert":
        X = phobert_encode(X, phobert, photokenizer)
    else:
        raise ValueError("Current supported text encoders: ['tfidf', 'sbert', 'phobert']")

    return X

In [60]:
X = {k: v["Sentence"].tolist() for k, v in preprocessed_data.items()}

In [64]:
embedding_types = ["tfidf", "sbert", "phobert"]
features = {k: {t: get_features(X[k], t) for t in embedding_types} for k, v in tqdm(X.items())}

  0%|                                                                                                           | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/26 [00:00<?, ?it/s]

 33%|█████████████████████████████████                                                                  | 1/3 [00:05<00:10,  5.07s/it]

Batches:   0%|          | 0/23 [00:00<?, ?it/s]

 67%|██████████████████████████████████████████████████████████████████                                 | 2/3 [00:09<00:04,  4.47s/it]

Batches:   0%|          | 0/29 [00:00<?, ?it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:14<00:00,  4.74s/it]


## Encode Labels

In [65]:
f = lambda x: [row[row == 1].index.tolist() for _, row in x.iterrows()]
y = {k: f(v) for k, v in preprocessed_data.items()}
label_encoders = {k: MultiLabelBinarizer().fit(v) for k, v in y.items()}
y = {k: label_encoders[k].transform(v) for k, v in y.items()}

## Training

In [ ]:
def make_model(embs_type):
    if embs_type == "tifdf":
        pipeline = Pipeline(steps=[
            ("vect", tfidf),
            ("clf", LogisticRegression())
        ])
    elif embs_type == "sbert":
        pipeline = Pipeline(steps=[
            ("vect", FunctionTransformer(sbert.encode)),
            ("clf", LogisticRegression())
        ])
    elif embs_type == "phobert":
        pipeline = Pipeline(steps=[
            ("vect", FunctionTransformer()),
            ("clf", LogisticRegression())
        ])
    else:
        raise ValueError("Current supported text encoders: ['tfidf', 'sbert', 'phobert']")
    
    return pipeline

In [ ]:
models = {for k, v in }

## Evaluate